# MNIST image classification with Swin Transformers 

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D

In [2]:
import sys
sys.path.append('../')

from keras_vision_transformer import swin_layers
from keras_vision_transformer import transformer_layers

# Data and problem statement

The MNIST dataset contains handwritten digits as gray-scale images with pixel sizes of 28-by-28. The pixel values are converted to float numbers and normalized with minimum-maximum scaling. The dataset is labeled with ten categories, represents digits of 0-9.

A supervised image classification problem is proposed to demonstrate the application of the Swin Transformer. By taking preprocessed grayscale images as inputs, the Swin Transformer is trained to classify the ten image labels. 

In [3]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# The Swin Transformer

Liu, Z., Lin, Y., Cao, Y., Hu, H., Wei, Y., Zhang, Z., Lin, S. and Guo, B., 2021. Swin transformer: Hierarchical vision transformer using shifted windows. arXiv preprint arXiv:2103.14030. https://arxiv.org/abs/2103.14030

## Hyperparameters

A simplified Swin Transformer configuration is applied with a patch embedding layer, two transformer blocks, and a patch merging layer. Global average pooling and softmax output activations are applied after Swin Transformer blocks.

See Liu et al. (2021) for more complicated architecture variants.

The hyperparameters of the Swin Transfor are listed as follows:

In [4]:
input_size = (28, 28, 1) # The image size of the MNIST
patch_size = (2, 2) # Segment 28-by-28 frames into 2-by-2 sized patches, patch contents and positions are embedded
n_labels = 10 # MNIST labels

# Dropout parameters
mlp_drop_rate = 0.01 # Droupout after each MLP layer
attn_drop_rate = 0.01 # Dropout after Swin-Attention
proj_drop_rate = 0.01 # Dropout at the end of each Swin-Attention block, i.e., after linear projections
drop_path_rate = 0.01 # Drop-path within skip-connections

# Self-attention parameters 
# (Fixed for all the blocks in this configuration, but can vary per block in larger architectures)
num_heads = 8 # Number of attention heads
embed_dim = 64 # Number of embedded dimensions
num_mlp = 256 # Number of MLP nodes
qkv_bias = True # Convert embedded patches to query, key, and values with a learnable additive value
qk_scale = None # None: Re-scale query based on embed dimensions per attention head # Float for user specified scaling factor

# Shift-window parameters
window_size = 2 # Size of attention window (height = width)
shift_size = window_size // 2 # Size of shifting (shift_size < window_size)

num_patch_x = input_size[0]//patch_size[0]
num_patch_y = input_size[1]//patch_size[1]

## Model configuration

In [5]:
# The input section
IN = Input(input_size)
X = IN

# Extract patches from the input tensor
X = transformer_layers.patch_extract(patch_size)(X)

# Embed patches to tokens
X = transformer_layers.patch_embedding(num_patch_x*num_patch_y, embed_dim)(X)

# -------------------- Swin transformers -------------------- #
# Stage 1: window-attention + Swin-attention + patch-merging

for i in range(2):
    
    if i % 2 == 0:
        shift_size_temp = 0
    else:
        shift_size_temp = shift_size

    X = swin_layers.SwinTransformerBlock(dim=embed_dim, num_patch=(num_patch_x, num_patch_y), num_heads=num_heads, 
                             window_size=window_size, shift_size=shift_size_temp, num_mlp=num_mlp, qkv_bias=qkv_bias, qk_scale=qk_scale,
                             mlp_drop=mlp_drop_rate, attn_drop=attn_drop_rate, proj_drop=proj_drop_rate, drop_path_prob=drop_path_rate, 
                             name='swin_block{}'.format(i))(X)
# Patch-merging
#    Pooling patch sequences. Half the number of patches (skip every two patches) and double the embedded dimensions
X = transformer_layers.patch_merging((num_patch_x, num_patch_y), embed_dim=embed_dim, name='down{}'.format(i))(X)

# ----------------------------------------------------------- #

# Convert embedded tokens (2D) to vectors (1D)
X = GlobalAveragePooling1D()(X)

# The output section
OUT = Dense(n_labels, activation='softmax')(X)

In [6]:
# Model configuration
model = keras.models.Model(inputs=[IN,], outputs=[OUT,])

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
patch_extract (patch_extract (None, 196, 4)            0         
_________________________________________________________________
patch_embedding (patch_embed (None, 196, 64)           12864     
_________________________________________________________________
swin_transformer_block (Swin (None, 196, 64)           50072     
_________________________________________________________________
swin_transformer_block_1 (Sw (None, 196, 64)           50856     
_________________________________________________________________
patch_merging (patch_merging (None, 49, 128)           32768     
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0     

## Training

Gradient clipping is applied to prevent gradient explosion.

Note: the traning of this example is not systematic, and is provided for illustration purposes only.

In [8]:
# Compile the model
opt = keras.optimizers.Adam(learning_rate=1e-4, clipvalue=0.5)
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=opt, metrics=['accuracy',])

# Training
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_split=0.1)

Epoch 1/5
1688/1688 [==============================] - 18s 9ms/step - loss: 0.9973 - accuracy: 0.6846 - val_loss: 0.2451 - val_accuracy: 0.9240
Epoch 2/5
1688/1688 [==============================] - 15s 9ms/step - loss: 0.2827 - accuracy: 0.9116 - val_loss: 0.1642 - val_accuracy: 0.9520
Epoch 3/5
1688/1688 [==============================] - 15s 9ms/step - loss: 0.2192 - accuracy: 0.9317 - val_loss: 0.1936 - val_accuracy: 0.9408
Epoch 4/5
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1768 - accuracy: 0.9469 - val_loss: 0.1270 - val_accuracy: 0.9620
Epoch 5/5
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1627 - accuracy: 0.9491 - val_loss: 0.1053 - val_accuracy: 0.9705


## Evaluation

In [9]:
y_pred = model.predict(x_train[:10, ...])

In [10]:
y_pred[0]

array([1.6543670e-07, 1.8593454e-07, 1.4957112e-05, 8.7632787e-01,
       3.2536024e-10, 1.2359164e-01, 1.2831727e-09, 1.5645363e-05,
       3.7107464e-05, 1.2507793e-05], dtype=float32)

## Save and reuse

The model can be saved as `model.save()`, but it contains python objects that are not part of the `tensorflow.keras`. Thus when loading the model, it is preferred to load the weights only, and freeze them within a new configuration.

e.g.

```python
weights = dummy_loader(model_old_path)
model_new = swin_transformer_model(...)
model_new.set_weights(weights)
```